# Presentación 17:

En esta presentación veremos cómo ajustar y realizar pronósticos con un modelo $SARIMA(p, d, q)\times(P, D, Q)_s$ en Python.

Además, veremos el equivalente a auto.arima del R.

Instalemos primero el módulo __pmdarima__ para utilizar la función __auto_arima__. El sitio web del módulo está [AQUÍ](https://pypi.org/project/pmdarima/). El siguiente código solo se ejecuta una vez y, con ello, el módulo ya está listo para usarse (no debe ejecutare ya más):

In [ ]:
!pip install pmdarima

In [ ]:
import pmdarima as pm
import numpy as np # Para trabajar con vectores y matrices
from matplotlib import pyplot as plt # Para graficar
import pandas as pd # Para trabajar con bases de datos
from statsmodels.tsa.arima.model import ARIMA
# from statsmodels.tsa.arima_model import ARIMA # Para cambiar el número maxiter.
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
plt.style.use('fivethirtyeight') # Para dar un estilo distinto a los gráficos

In [ ]:
SARIMAX?

In [ ]:
pm.auto_arima?

In [ ]:
direccion = "../../DATOS/anexo_empleo_ago_22.xlsx"
bd_original = pd.read_excel(direccion, sheet_name="Total nacional",
                           skiprows=12)

In [ ]:
bd_original.head()

In [ ]:
bd_original.shape

In [ ]:
bd_original.iloc[3,1:261].to_frame().reset_index().iloc[:,1].to_frame().rename(columns={3:"td"})

In [ ]:
tasa_desemp=bd_original.iloc[3,1:261].to_frame().reset_index().iloc[:,1].to_frame().rename(columns={3:"td"})

In [ ]:
tasa_desemp.dtypes

In [ ]:
tasa_desemp.td = tasa_desemp.td.astype("float")

In [ ]:
pd.date_range(start="2001-01-1",end="2022-09-01", freq="M")

In [ ]:
tasa_desemp["fecha"]=pd.date_range(start="2001-01-1",end="2022-09-01", freq="M")

In [ ]:
tasa_desemp

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(tasa_desemp.fecha, tasa_desemp.td)
plt.show()

## Actividad:

1. Realice los gráficos ACF y PACF para tratar de identificar los órdenes del modelo $SARIMA(p, d, q)\times(P,D,Q)_s$.

2. Aplique la prueba de Dickey-Fuller para ver si es necesario tomar diferencias (valor de $d$). 

3. Aplique varios modelos y use un criterio de selección (AIC, BIC) para seleccionar el "mejor".

4. Verifique si existe autocorrelación en los residuales del modelo seleccionado utilizando el gráfico ACF y complementando con la prueba de Ljung-Box.

5. Verifique si hay normalidad en los residuales del modelo.

6. Si todo lo anterior se cumple entonces puede realizar pronósticos con el "mejor" modelo ajustado.

7. Realice el mismo procedimiento para la serie de la BD USgas.csv

In [ ]:
modelo1 = SARIMAX(tasa_desemp.td, order=(0, 1, 0),
    seasonal_order=(0, 0, 4, 6)).fit()

In [ ]:
modelo1.summary()

In [ ]:
modelo1.summary().tables[1]

In [ ]:
modelo1.plot_diagnostics(figsize=(15, 12))
plt.show()

Como era de esperarse los residuales son no correlacionados (se ve en el gráfico de la ACF con todos los valores dentro de la banda de confianza), oscilan alrededor del cero con varianza constante (primer gráfico) y se observa gráficamente que siguen una distribución normal.

6. Realizamos entonces pronósticos. Existen dos funciones o procesos: __get_prediction__ (permite hacer predicciones de toda la serie y del futuro de la misma desde un punto inicial hasta un punto final) y __get_forecast__ (solo permite realizar predicciones futuras). En escencia el  __get_forecast__ es un caso particular del __get_prediction__.

In [ ]:
pred.conf_int?

In [ ]:
pred = modelo1.get_prediction(start=260, end=271, dynamic=False) # dynamic=False evita que las predicciones futuras
                                                               # se conviertan en datos de la muestra para predicciones 
                                                               # que van más adelante.
pred_ci = pred.conf_int(alpha=0.05) # Intervalo de confianza 
pred_ci

In [ ]:
pred.predicted_mean # Valores predichos a futuro.

In [ ]:
fore = modelo1.get_forecast(12) # Obtenemos lo mismo que con el proceso anterior
fore_ci = fore.conf_int()
fore_ci

In [ ]:
fore.predicted_mean # Valores predichos a futuro.

In [ ]:
td_pred = np.concatenate((tasa_desemp.td, fore.predicted_mean)) # Adicionamos a la serie original las predicciones.
td_pred

In [ ]:
td_pred[260:272]

In [ ]:
plt.figure(figsize=(16,8))  #  Graficamos:
plt.plot(td_pred, marker="o")
plt.plot(range(260,272,1),td_pred[260:273], marker="o")
plt.plot(range(260,272,1), pred_ci.iloc[:,0], marker="o", color="green")
plt.plot(range(260,272,1), pred_ci.iloc[:,1], marker="o", color="green")
plt.show()

## Ahora proponga varios modelos y seleccione el "mejor". Luego aplique la función auto_arima.

